In [3]:
import tensorflow
import pandas
import numpy 
import pickle
from tensorflow.keras.models import load_model

### Load the trained model, scaler and encoders pickle

In [4]:
model = load_model('model.h5')

with open('lable_encoder_gender.pkl', 'rb') as file:
    lable_encoder_gender = pickle.load(file)

with open('onehot_encoder_geo.pkl', 'rb') as file:
    onehot_encoder_geo = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [5]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': "Male",
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [6]:
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pandas.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

c:\Users\User\Desktop\Coringa\AI Generative Course\Deep Learning\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [7]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [8]:
input_data_df = pandas.DataFrame([input_data])

In [9]:
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [10]:
input_data_df = pandas.concat([input_data_df.drop(['Geography'], axis=1), geo_encoded_df], axis=1)

In [11]:
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [12]:
input_data_df['Gender'] = lable_encoder_gender.transform(input_data_df['Gender'])

In [13]:
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [14]:
input_data_df_scaled = scaler.transform(input_data_df)
input_data_df_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

### Predict Churn

In [15]:
prediction = model.predict(input_data_df_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step


In [16]:
prediction

array([[0.04601367]], dtype=float32)

In [18]:
prediction_proba = prediction[0][0]

In [19]:
prediction_proba

np.float32(0.046013672)

In [20]:
if prediction_proba > 0.5:
    print('The customer is likely to churn')
else:
    print('The customer is not likely to churn')

The customer is not likely to churn
